In [2]:
import io
import numpy as np
import tensorflow as tf
from librosa import effects

from utils import spectrogram2wav, plot_test_alignment

from train import Graph
from hyperparams import Hyperparams as hp
import tqdm

In [3]:
def load_vocab():
    char2idx = {char: idx for idx, char in enumerate(hp.vocab)}
    idx2char = {idx: char for idx, char in enumerate(hp.vocab)}
    return char2idx, idx2char

In [4]:
def get_EOS_index(text):
    # Load vocab
    char2idx, idx2char = load_vocab()

    _text = np.array([idx2char[t] for t in text])
    return np.argmax(_text == hp.EOS_char)

In [5]:
char2idx, idx2char = load_vocab()

In [6]:
idx2char

{0: '␀',
 1: '␃',
 2: ' ',
 3: '!',
 4: "'",
 5: ',',
 6: '-',
 7: '.',
 8: ':',
 9: ';',
 10: '?',
 11: 'A',
 12: 'B',
 13: 'C',
 14: 'D',
 15: 'E',
 16: 'F',
 17: 'G',
 18: 'H',
 19: 'I',
 20: 'J',
 21: 'K',
 22: 'L',
 23: 'M',
 24: 'N',
 25: 'O',
 26: 'P',
 27: 'Q',
 28: 'R',
 29: 'S',
 30: 'T',
 31: 'U',
 32: 'V',
 33: 'W',
 34: 'X',
 35: 'Y',
 36: 'Z',
 37: 'a',
 38: 'b',
 39: 'c',
 40: 'd',
 41: 'e',
 42: 'f',
 43: 'g',
 44: 'h',
 45: 'i',
 46: 'j',
 47: 'k',
 48: 'l',
 49: 'm',
 50: 'n',
 51: 'o',
 52: 'p',
 53: 'q',
 54: 'r',
 55: 's',
 56: 't',
 57: 'u',
 58: 'v',
 59: 'w',
 60: 'x',
 61: 'y',
 62: 'z',
 63: '¡',
 64: '¿',
 65: 'Á',
 66: 'Å',
 67: 'É',
 68: 'Í',
 69: 'Ó',
 70: 'Ú',
 71: 'á',
 72: 'æ',
 73: 'è',
 74: 'é',
 75: 'ë',
 76: 'í',
 77: 'î',
 78: 'ñ',
 79: 'ó',
 80: 'ö',
 81: 'ú',
 82: 'ü',
 83: '—'}

In [7]:
char2idx

{'␀': 0,
 '␃': 1,
 ' ': 2,
 '!': 3,
 "'": 4,
 ',': 5,
 '-': 6,
 '.': 7,
 ':': 8,
 ';': 9,
 '?': 10,
 'A': 11,
 'B': 12,
 'C': 13,
 'D': 14,
 'E': 15,
 'F': 16,
 'G': 17,
 'H': 18,
 'I': 19,
 'J': 20,
 'K': 21,
 'L': 22,
 'M': 23,
 'N': 24,
 'O': 25,
 'P': 26,
 'Q': 27,
 'R': 28,
 'S': 29,
 'T': 30,
 'U': 31,
 'V': 32,
 'W': 33,
 'X': 34,
 'Y': 35,
 'Z': 36,
 'a': 37,
 'b': 38,
 'c': 39,
 'd': 40,
 'e': 41,
 'f': 42,
 'g': 43,
 'h': 44,
 'i': 45,
 'j': 46,
 'k': 47,
 'l': 48,
 'm': 49,
 'n': 50,
 'o': 51,
 'p': 52,
 'q': 53,
 'r': 54,
 's': 55,
 't': 56,
 'u': 57,
 'v': 58,
 'w': 59,
 'x': 60,
 'y': 61,
 'z': 62,
 '¡': 63,
 '¿': 64,
 'Á': 65,
 'Å': 66,
 'É': 67,
 'Í': 68,
 'Ó': 69,
 'Ú': 70,
 'á': 71,
 'æ': 72,
 'è': 73,
 'é': 74,
 'ë': 75,
 'í': 76,
 'î': 77,
 'ñ': 78,
 'ó': 79,
 'ö': 80,
 'ú': 81,
 'ü': 82,
 '—': 83}

In [8]:
[t for t in 'sd']

['s', 'd']

In [9]:
idx2char[40]

'd'

In [10]:
char2idx['d']

40

In [11]:
def get_EOS_index(text):
    # text : texto codificado en numeros
    # Load vocab
    char2idx, idx2char = load_vocab()

    _text = np.array([idx2char[t] for t in text])
    return np.argmax(_text == hp.EOS_char)

def get_EOS_fire(alignment,text):
    EOS_index = get_EOS_index(text)
    text_max_indicies = np.argmax(alignment,axis = 0)
    r = []
    for i,max_index in enumerate(text_max_indicies):
        if max_index == EOS_index:
            r.append(i)
    if not len(r) == 0:
        return max(r)
    return None

import scipy
def save_wav(wav, path, sr):
  wav *= 32767 / max(0.01, np.max(np.abs(wav)))
  scipy.io.wavfile.write(path, sr, wav.astype(np.int16))

In [12]:
g = Graph(mode="synthesize"); print("Graph loaded")

saver = tf.train.Saver()

session = tf.Session()
session.run(tf.global_variables_initializer())
saver = tf.train.Saver()
saver.restore(session, tf.train.latest_checkpoint(hp.syn_logdir)); print("Restored!")

Graph loaded
INFO:tensorflow:Restoring parameters from logdir_es\model_gs_400k
Restored!


In [13]:
char2idx, idx2char = load_vocab()

In [34]:
text = 'Hola mundo'

In [35]:
text_encode = [char2idx[char] for char in text]

In [16]:
text_encode

[18, 51, 48, 37, 2, 49, 57, 50, 40, 51]

In [17]:
y_hat = np.zeros((1, 200, hp.n_mels*hp.r), np.float32)

In [18]:
y_hat

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]], dtype=float32)

In [36]:

_y_hat = session.run(g.y_hat, {g.x: [text_encode], g.y: y_hat})
y_hat[:, j, :] = _y_hat[:, j, :]

In [37]:
## alignments
alignments = session.run([g.alignments], {g.x: [text_encode], g.y: y_hat})[0]
## mag
mag = session.run(g.z_hat, {g.y_hat: y_hat})

In [38]:
text, alignment = text[0], alignments[0]
print(alignment.shape)

(10, 200)


In [39]:
print("len text", float(len(text)))

len text 1.0


In [40]:
SEC_PER_CHAR = float(10)/180  #[sec/char]
SEC_PER_ITER = float(12)/200  #[sec/iter]
min_sample_sec = float(get_EOS_index(text_encode))*SEC_PER_CHAR#/SEC_PER_ITER
print("min sec ",min_sample_sec)

min sec  0.0


In [41]:
al_EOS_index = get_EOS_fire(alignment,text_encode)
al_EOS_index = None

In [42]:
a,b,c = mag.shape

In [43]:
mag.T.shape

(1025, 1000, 1)

In [44]:
mag.reshape(b,c).shape

(1000, 1025)

In [45]:
n_mag = mag.reshape(b,c)

In [46]:
audio = spectrogram2wav(n_mag,min_sample_sec)